In [ ]:
import pandas as pd
import numpy as np
import numpy as np
from transformers import BertTokenizerFast
from transformers import DataCollatorForTokenClassification
from transformers import AutoModelForTokenClassification
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizerFast

# Initialize the fast tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForTokenClassification, AdamW
from sklearn.preprocessing import LabelEncoder


In [ ]:
data = pd.read_parquet("new_data.parquet")

In [ ]:
data.head()

,term,postags,entitytags,sentence_id
0,Masked,VBN,O,0
1,assailants,NNS,O,0
2,with,IN,O,0
3,grenades,NNS,O,0
4,and,CC,O,0


In [ ]:
le = LabelEncoder()
le1 = LabelEncoder()

data['postags'] = le.fit_transform(data['postags'])
data['entitytags'] = le1.fit_transform(data['entitytags'])

grouped_df = data.groupby('sentence_id').agg({
    'term': list,
    'postags': list,
    'entitytags': list
}).reset_index()

# Display the result
grouped_df.head()

,sentence_id,term,postags,entitytags
0,0,"[Masked, assailants, with, grenades, and, auto...","[34, 19, 10, 19, 5, 11, 19, 32, 7, 33, 16, 10,...","[16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 1..."
1,1,"[Turkish, officials, said, the, attack, occurr...","[11, 19, 32, 7, 16, 32, 17, 10, 7, 16, 10, 17,...","[3, 16, 16, 16, 16, 16, 7, 16, 16, 16, 16, 2, ..."
2,2,"[The, wounded, were, taken, to, the, hospital,...","[7, 34, 32, 34, 29, 7, 16, 10, 7, 11, 16, 10, ...","[16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 1..."
3,3,"[Soldiers, cut, off, the, roads, leading, to, ...","[19, 32, 27, 7, 19, 33, 29, 7, 16, 10, 22, 32,...","[16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 1..."
4,4,"[Officials, said, they, were, not, sure, of, t...","[19, 32, 22, 32, 24, 11, 10, 7, 16, 10, 7, 16,...","[16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 1..."


In [ ]:
sentences = grouped_df['term'].tolist()  # Each element is a list of words
pos_tags = grouped_df['postags'].tolist()   # Each element is a list of tags

In [ ]:
# Check for None values
if any(s is None for s in sentences) or any(t is None for t in pos_tags):
    raise ValueError("Found None in sentences or pos_tags")

In [ ]:
sentences.pop(10924)
pos_tags.pop(10924)

sentences.pop(15322)
pos_tags.pop(15322)

In [ ]:
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Split the data into training and validation sets
train_sentences, val_sentences, train_tags, val_tags = train_test_split(
    sentences, pos_tags, test_size=0.1, random_state=42
)

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

def tokenize_and_preserve_labels(sentence, text_labels):
    tokenized_sentence = []
    labels = []

    # Check if sentence and text_labels are lists
    if not isinstance(sentence, list) or not isinstance(text_labels, list):
        raise ValueError(f"Expected lists, got {type(sentence)} and {type(text_labels)}")

    for word, label in zip(sentence, text_labels):  # Ensure these are lists of tokens/tags
        tokenized_word = tokenizer.tokenize(word)
        tokenized_sentence.extend(tokenized_word)

        # Extend the label to all sub-tokens
        labels.extend([label] * len(tokenized_word))

    return tokenized_sentence, labels

# Add try-except to catch and print issues during tokenization
tokenized_texts_and_labels = []
for sent, labs in zip(train_sentences, train_tags):
    try:
        tokenized_texts_and_labels.append(tokenize_and_preserve_labels(sent, labs))
    except Exception as e:
        print(f"Error processing sentence: {sent} with labels: {labs}")
        print(f"Error: {e}")
        raise e

tokenized_texts = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels]
labels = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels]


In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForTokenClassification, AdamW

# Set the maximum sequence length
MAX_LEN = 75
batch_size = 16

input_ids = [tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", value=0.0, truncating="post", padding="post")

# Convert labels to numeric values
tag_values = list(set(tag for doc in labels for tag in doc))
tag_values.append("PAD")
tag2idx = {t: i for i, t in enumerate(tag_values)}

tags = [[tag2idx.get(l) for l in lab] for lab in labels]
tags = pad_sequences(tags, maxlen=MAX_LEN, dtype="long", value=tag2idx["PAD"], truncating="post", padding="post")

attention_masks = [[float(i != 0.0) for i in ii] for ii in input_ids]

# Convert to torch tensors
train_inputs = torch.tensor(input_ids)
train_tags = torch.tensor(tags)
train_masks = torch.tensor(attention_masks)

# Create DataLoader
train_data = TensorDataset(train_inputs, train_masks, train_tags)
train_sampler = RandomSampler(train_data)
train_dataloa§r = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = BertForTokenClassification.from_pretrained(
    "bert-base-cased",
    num_labels=len(tag2idx),
    output_attentions=False,
    output_hidden_states=False
).to(device)

optimizer = AdamW(model.parameters(), lr=3e-5, eps=1e-8)

epochs = 4
for epoch in range(epochs):
    model.train()
    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        # Clear previously calculated gradients
        model.zero_grad()

        # Forward pass
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs.loss

        # Backward pass
        loss.backward()

        # Update parameters
        optimizer.step()
        if step % 5 == 0:
            print(f"Epoch {epoch + 1}, Step {step}, Loss {loss.item()}")


In [ ]:
# Tokenize the validation data
val_tokenized_texts_and_labels = [
    tokenize_and_preserve_labels(sent, labs)
    for sent, labs in zip(val_sentences, val_tags)
]

val_tokenized_texts = [token_label_pair[0] for token_label_pair in val_tokenized_texts_and_labels]
val_labels = [token_label_pair[1] for token_label_pair in val_tokenized_texts_and_labels]

val_input_ids = [tokenizer.convert_tokens_to_ids(txt) for txt in val_tokenized_texts]
val_input_ids = pad_sequences(val_input_ids, maxlen=MAX_LEN, dtype="long", value=0.0, truncating="post", padding="post")

val_tags = [[tag2idx.get(l) for l in lab] for lab in val_labels]
val_tags = pad_sequences(val_tags, maxlen=MAX_LEN, dtype="long", value=tag2idx["PAD"], truncating="post", padding="post")

val_attention_masks = [[float(i != 0.0) for i in ii] for ii in val_input_ids]

# Convert to torch tensors
val_inputs = torch.tensor(val_input_ids)
val_tags = torch.tensor(val_tags)
val_masks = torch.tensor(val_attention_masks)

# Create DataLoader
val_data = TensorDataset(val_inputs, val_masks, val_tags)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)


In [ ]:
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

# Put model in evaluation mode
model.eval()

# Tracking variables
predictions, true_labels = [], []

for batch in val_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

    logits = outputs.logits
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
    true_labels.extend(label_ids)

# Flatten predictions and true labels
pred_tags = [tag_values[p_i] for p in predictions for p_i, l_i in zip(p, p) if tag_values[l_i] != "PAD"]
valid_tags = [tag_values[l_i] for l in true_labels for l_i in l if tag_values[l_i] != "PAD"]


In [ ]:
print(len(valid_tags), len(pred_tags))

51850 51850


In [ ]:
# Calculate accuracy
accuracy = accuracy_score(valid_tags, pred_tags)
print(f"Validation Accuracy: {accuracy}")


# Classification report
report = classification_report(valid_tags, pred_tags)
print("Validation Classification Report:")
print(report)


Validation Accuracy: 0.9848987463837994
Validation Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        47
           1       1.00      1.00      1.00      1228
           2       1.00      1.00      1.00      1825
           3       0.75      0.89      0.81        46
           4       1.00      1.00      1.00         7
           5       1.00      0.99      1.00       899
           6       0.99      0.99      0.99      1251
           7       1.00      1.00      1.00      3733
           8       0.96      0.96      0.96        27
          10       1.00      0.99      0.99      4673
          11       0.96      0.95      0.95      3976
          12       0.96      0.93      0.95       107
          13       0.99      0.93      0.96       124
          14       1.00      1.00      1.00        35
          15       1.00      1.00      1.00       286
          16       0.98      0.97      0.98      6081
       

#NER

In [ ]:
df = pd.read_csv('ner_cleaned.csv')

In [ ]:
df.head()

,Word,POS,Tag
0,Thousands,NNS,O
1,of,IN,O
2,demonstrators,NNS,O
3,have,VBP,O
4,marched,VBN,O


In [ ]:
le2 = LabelEncoder()
le3 = LabelEncoder()

df['POS'] = le2.fit_transform(df['POS'])
df['Tag'] = le3.fit_transform(df['Tag'])

In [ ]:
import ast

# Sample data transformation functions
def to_list(val):
    try:
        # Attempt to convert a string representation of a list to an actual list
        return ast.literal_eval(val) if isinstance(val, str) else val
    except (ValueError, SyntaxError):
        return None

# Ensure sentences are lists of words
df['sentence'] = df['Word'].apply(lambda x: x.split() if isinstance(x, str) else x)

# Convert pos_tags and ner_tags from int to list
df['pos_tags'] = df['POS'].apply(lambda x: [x] if isinstance(x, int) else x)
df['ner_tags'] = df['Tag'].apply(lambda x: [x] if isinstance(x, int) else x)

# Filter out rows where any column is not properly formatted
df = df.dropna(subset=['sentence', 'pos_tags', 'ner_tags'])

# Verify the remaining data
print(f"Number of valid samples: {len(df)}")


Number of valid samples: 923624


In [ ]:
from sklearn.model_selection import train_test_split

sentences = df['sentence'].tolist()
pos_tags = df['pos_tags'].tolist()
ner_tags = df['ner_tags'].tolist()

# Split the data into training and validation sets
train_sentences, val_sentences, train_pos_tags, val_pos_tags, train_ner_tags, val_ner_tags = train_test_split(
    sentences, pos_tags, ner_tags, test_size=0.1, random_state=42
)

# Ensure the split was successful
print(f"Training samples: {len(train_sentences)}, Validation samples: {len(val_sentences)}")


Training samples: 831261, Validation samples: 92363


In [ ]:
# Extract unique tags from the data
unique_pos_tags = set(tag for tags in df['pos_tags'] for tag in tags)
unique_ner_tags = set(tag for tags in df['ner_tags'] for tag in tags)

# Create mappings
pos_tag2idx = {tag: idx for idx, tag in enumerate(unique_pos_tags)}
pos_tag2idx['PAD'] = len(pos_tag2idx)  # Add a PAD token

ner_tag2idx = {tag: idx for idx, tag in enumerate(unique_ner_tags)}
ner_tag2idx['PAD'] = len(ner_tag2idx)  # Add a PAD token

# Verify the mappings
print("POS tag to index mapping:", pos_tag2idx)
print("NER tag to index mapping:", ner_tag2idx)


POS tag to index mapping: {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13, 14: 14, 15: 15, 16: 16, 17: 17, 18: 18, 19: 19, 20: 20, 21: 21, 22: 22, 23: 23, 24: 24, 25: 25, 26: 26, 27: 27, 28: 28, 29: 29, 30: 30, 31: 31, 'PAD': 32}
NER tag to index mapping: {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13, 14: 14, 15: 15, 16: 16, 'PAD': 17}


In [ ]:
from transformers import BertTokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

def tokenize_and_preserve_labels(sentence, pos_labels, ner_labels):
    tokenized_sentence = []
    tokenized_pos_labels = []
    tokenized_ner_labels = []

    for word, pos, ner in zip(sentence, pos_labels, ner_labels):
        tokenized_word = tokenizer.tokenize(word)
        tokenized_sentence.extend(tokenized_word)
        tokenized_pos_labels.extend([pos] * len(tokenized_word))
        tokenized_ner_labels.extend([ner] * len(tokenized_word))

    return tokenized_sentence, tokenized_pos_labels, tokenized_ner_labels

# Tokenize and align labels
tokenized_texts_and_labels = []
for sent, pos, ner in zip(train_sentences, train_pos_tags, train_ner_tags):
    try:
        tokenized_texts_and_labels.append(tokenize_and_preserve_labels(sent, pos, ner))
    except Exception as e:
        print(f"Error processing sentence: {sent} with POS labels: {pos} and NER labels: {ner}")
        print(f"Error: {e}")
        raise e

tokenized_texts = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels]
tokenized_pos_tags = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels]
tokenized_ner_tags = [token_label_pair[2] for token_label_pair in tokenized_texts_and_labels]

# Pad sequences
MAX_LEN = 75

input_ids = [tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", value=0.0, truncating="post", padding="post")

pos_tags = [[pos_tag2idx.get(l) for l in lab] for lab in tokenized_pos_tags]
pos_tags = pad_sequences(pos_tags, maxlen=MAX_LEN, dtype="long", value=pos_tag2idx["PAD"], truncating="post", padding="post")

ner_tags = [[ner_tag2idx.get(l) for l in lab] for lab in tokenized_ner_tags]
ner_tags = pad_sequences(ner_tags, maxlen=MAX_LEN, dtype="long", value=ner_tag2idx["PAD"], truncating="post", padding="post")

attention_masks = [[float(i != 0.0) for i in ii] for ii in input_ids]

# Convert to tensors
import torch

train_inputs = torch.tensor(input_ids)
train_pos_tags = torch.tensor(pos_tags)
train_ner_tags = torch.tensor(ner_tags)
train_masks = torch.tensor(attention_masks)

# Create DataLoader
from torch.utils.data import DataLoader, TensorDataset, RandomSampler

batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_pos_tags, train_ner_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)


In [ ]:
from transformers import BertForTokenClassification, AdamW

# Load the BERT model for token classification
model = BertForTokenClassification.from_pretrained(
    'bert-base-cased',
    num_labels=len(ner_tag2idx),  # Set the number of labels for the NER task
    output_attentions=False,
    output_hidden_states=False
)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [ ]:
# Define the optimizer
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)

# Total number of training steps is [number of batches] x [number of epochs]
epochs = 1
total_steps = len(train_dataloader) * epochs

# Set up the learning rate scheduler
from transformers import get_linear_schedule_with_warmup

scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=total_steps)

# Define the loss function
loss_fn = torch.nn.CrossEntropyLoss()


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
import numpy as np
from tqdm import tqdm

# Function to calculate accuracy
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=2).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# Training loop
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")

    # Training phase
    model.train()
    total_loss = 0

    for step, batch in enumerate(tqdm(train_dataloader, desc="Training")):
        # Unpack the batch and load onto GPU if available
        b_input_ids, b_input_mask, b_labels_pos, b_labels_ner = tuple(t.to(device) for t in batch)

        # Clear previously calculated gradients
        model.zero_grad()

        # Forward pass
        outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels_ner)
        loss = outputs.loss

        # Accumulate the training loss
        total_loss += loss.item()

        # Backward pass to calculate gradients
        loss.backward()

        # Clip the norm of the gradients to prevent exploding gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        # Update parameters and learning rate
        optimizer.step()
        scheduler.step()

    # Calculate the average loss over the training data
    avg_train_loss = total_loss / len(train_dataloader)
    print(f"Average training loss: {avg_train_loss}")

    # Validation phase
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps = 0

    for batch in tqdm(train_dataloader, desc="Validation"):
        b_input_ids, b_input_mask, b_labels_pos, b_labels_ner = tuple(t.to(device) for t in batch)

        with torch.no_grad():
            outputs = model(b_input_ids, attention_mask=b_input_mask)

        logits = outputs.logits.detach().cpu().numpy()
        label_ids = b_labels_ner.to('cpu').numpy()

        eval_accuracy += flat_accuracy(logits, label_ids)
        nb_eval_steps += 1

    print(f"Validation Accuracy: {eval_accuracy / nb_eval_steps}")
